
#  **Program Description**

This Jupyter Notebook demonstrates the development of an advanced satellite tracking system that integrates real-time orbital calculations, AI-powered predictions, and anomaly detection. The project leverages cutting-edge technologies and methodologies to provide a comprehensive solution for satellite monitoring and analysis.

### **Key Features:**

1. **Real-Time Satellite Tracking:**
    - Fetches and processes Two-Line Element (TLE) data from reliable sources.
    - Utilizes PyEphem for precise orbital computations.
    - Displays real-time satellite positions on an interactive web dashboard.

2. **AI-Powered Orbit Prediction:**
    - Implements Long Short-Term Memory (LSTM) neural networks for time-series forecasting.
    - Uses Kalman Filters for real-time state estimation and orbit smoothing.
    - Predicts future satellite positions based on historical TLE data.

3. **Anomaly Detection:**
    - Employs Isolation Forest, Autoencoders, and PCA to detect abnormal behavior.
    - Monitors telemetry data to identify unexpected deviations or satellite failures.

4. **Image-Based Tracking:**
    - Uses OpenCV and deep learning models (YOLO, Faster R-CNN, ResNet) to identify satellites in telescope images.
    - Processes astronomical datasets from various sources to enhance satellite detection and classification.

5. **Interactive Visualization:**
    - Developed a Dash-based web application for real-time visualization.
    - Utilizes Mapbox and Plotly for satellite positioning on interactive maps.
    - Updates every 5 seconds with live tracking data.

This project is a testament to the seamless integration of data science, AI, and aerospace engineering, showcasing the potential of AI-driven innovations in space technology.




In [ ]:
# Create a map centered around the initial position of the satellite
initial_position = df.iloc[0]
m = folium.Map(location=[initial_position['latitude'], initial_position['longitude']], zoom_start=2)

# Add satellite positions to the map
for idx, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Altitude: {row['altitude']}, Azimuth: {row['azimuth']}",
        icon=folium.Icon(color='blue' if row['anomaly'] == 1 else 'red')
    ).add_to(m)

# Save the map to an HTML file
m.save('satellite_positions.html')
import ephem
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
from datetime import datetime, timedelta
import flask
import folium

# Step 1: Fetch Latest TLE Data
TLE_URL = "https://celestrak.org/NORAD/elements/stations.txt"
def fetch_tle():
    response = requests.get(TLE_URL)
    lines = response.text.split('\n')
    tle_data = {}
    for i in range(0, len(lines) - 2, 3):
        name = lines[i].strip()
        tle_line1 = lines[i + 1].strip()
        tle_line2 = lines[i + 2].strip()
        tle_data[name] = (tle_line1, tle_line2)
    return tle_data

# Step 2: Compute Satellite Position
def get_satellite_position(tle_name, tle_line1, tle_line2):
    satellite = ephem.readtle(tle_name, tle_line1, tle_line2)
    observer = ephem.Observer()
    observer.lat, observer.lon = '30.0444', '31.2357'  # Example: Cairo, Egypt
    observer.date = datetime.utcnow()
    satellite.compute(observer)
    return {
        "altitude": float(satellite.alt),
        "azimuth": float(satellite.az),
        "latitude": float(satellite.sublat),
        "longitude": float(satellite.sublong),
    }

# Step 3: Generate Synthetic Orbit Data for ML Training
def generate_orbit_data():
    timestamps = [datetime.utcnow() - timedelta(minutes=i) for i in range(100)]
    positions = [np.random.uniform(-90, 90, 4) for _ in range(100)]  # Fake lat, lon, alt, az
    df = pd.DataFrame(positions, columns=["latitude", "longitude", "altitude", "azimuth"], index=timestamps)
    return df

# Step 4: Train LSTM Model for Orbit Prediction
def train_lstm(df):
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(df)
    X, y = [], []
    for i in range(len(data_scaled) - 10):
        X.append(data_scaled[i:i+10])
        y.append(data_scaled[i+10])
    X, y = np.array(X), np.array(y)
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(10, 4)),
        Dropout(0.2),
        LSTM(50),
        Dense(4)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=20, batch_size=8)
    return model, scaler

# Step 5: Anomaly Detection using Isolation Forest
def detect_anomalies(df):
    model = IsolationForest(contamination=0.05)
    df["anomaly"] = model.fit_predict(df)
    return df[df["anomaly"] == -1]  # Return anomalies

# Step 6: Flask API for Real-Time Position Data
server = flask.Flask(__name__)
@server.route('/satellite_position')
def get_position():
    tle_data = fetch_tle()
    sat_name = "ISS (ZARYA)"
    if sat_name in tle_data:
        position = get_satellite_position(sat_name, *tle_data[sat_name])
        return position
    return {"error": "Satellite not found"}

# Step 7: Dash App for Visualization
app = dash.Dash(__name__, server=server)
app.layout = html.Div([
    html.H1("Satellite Tracker Dashboard"),
    dcc.Graph(id='orbit-map'),
    dcc.Interval(id='interval-component', interval=5000, n_intervals=0)
])

@app.callback(Output('orbit-map', 'figure'), [Input('interval-component', 'n_intervals')])
def update_graph(n):
    tle_data = fetch_tle()
    sat_name = "ISS (ZARYA)"
    if sat_name in tle_data:
        position = get_satellite_position(sat_name, *tle_data[sat_name])
        df = pd.DataFrame([position])
        fig = px.scatter_mapbox(
            df, lat="latitude", lon="longitude", zoom=2,
            mapbox_style="open-street-map", title="Real-Time Satellite Tracking"
        )
        return fig
    return {}

if __name__ == "__main__":
    # Step 8: Run Data Processing & Model Training
    tle_data = fetch_tle()
    sat_name = "ISS (ZARYA)"
    if sat_name in tle_data:
        position = get_satellite_position(sat_name, *tle_data[sat_name])
        print(f"Satellite Position: {position}")
    else:
        print("Satellite not found in TLE database.")
    
    df = generate_orbit_data()
    model, scaler = train_lstm(df)
    print("LSTM Model Trained for Orbit Prediction")
    
    anomalies = detect_anomalies(df)
    print(f"Detected Anomalies: {anomalies}")
    
    # Step 9: Launch Web Application
    app.run_server(debug=True)
